In [2]:
import pandas as pd
from pathlib import Path

In [11]:
data_folder = Path("Data/")
csv_paths = sorted(data_folder.glob("*.csv"))

dfs = {}
for path in csv_paths:
    name = path.stem
    df = pd.read_csv(path)
    dfs[name] = df
    print(f"Loaded '{name}' with shape {df.shape}")

Loaded '2023_NO2_IT' with shape (11951, 27)
Loaded '2023_O3_IT' with shape (10077, 27)
Loaded '2023_PM10_IT' with shape (8228, 27)
Loaded '2023_PM25_IT' with shape (3573, 27)
Loaded '2024_NO2_IT' with shape (11875, 27)
Loaded '2024_O3_IT' with shape (10075, 27)
Loaded '2024_PM10_IT' with shape (8384, 27)
Loaded '2024_PM25_IT' with shape (3704, 27)


### Visualize First Few Rows of Each CSV

In [16]:
df_NO2_2023 = dfs["2023_NO2_IT"]
df_O3_2023 = dfs["2023_O3_IT"]
df_PM10_2023 = dfs["2023_PM10_IT"]
df_PM25_2023 = dfs["2023_PM25_IT"]
df_NO2_2024 = dfs["2024_NO2_IT"]
df_O3_2024 = dfs["2024_O3_IT"]
df_PM10_2024 = dfs["2024_PM10_IT"]
df_PM25_2024 = dfs["2024_PM25_IT"]
print(df_NO2_2023.head(), "\n" , df_O3_2023.head(), "\n" , df_PM10_2023.head(), "\n" , df_PM25_2023.head(), "\n" , df_NO2_2024.head(), "\n" , df_O3_2024.head(), "\n" , df_PM10_2024.head(), "\n" , df_PM25_2024.head())

  Country Air Quality Network                 Air Quality Network Name  \
0   Italy          NET.IT280A  RETE MONITORAGGIO ARIA REGIONE SARDEGNA   
1   Italy          NET.IT280A  RETE MONITORAGGIO ARIA REGIONE SARDEGNA   
2   Italy          NET.IT280A  RETE MONITORAGGIO ARIA REGIONE SARDEGNA   
3   Italy          NET.IT280A  RETE MONITORAGGIO ARIA REGIONE SARDEGNA   
4   Italy          NET.IT280A  RETE MONITORAGGIO ARIA REGIONE SARDEGNA   

  Air Quality Station EoI Code Air Quality Station Name  \
0                      IT2040A                   CENQU1   
1                      IT2008A                   CESGI1   
2                      IT2040A                   CENQU1   
3                      IT2010A                   CENPT1   
4                      IT2010A                   CENPT1   

                         Sampling Point Id Air Pollutant  \
0  SPO.IT2040A_8_chemi_2011-05-19_00:00:00           NO2   
1  SPO.IT2008A_8_chemi_2011-02-17_00:00:00           NO2   
2  SPO.IT2040A_8_che

In [18]:
print(df_NO2_2023.info(), "\n" , df_O3_2023.info(), "\n" , df_PM10_2023.info(), "\n" , df_PM25_2023.info(), "\n" , df_NO2_2024.info(), "\n" , df_O3_2024.info(), "\n" , df_PM10_2024.info(), "\n" , df_PM25_2024.info())
print("Df describe details:"+ "\n" + "\n")
print(df_NO2_2023.describe(), "\n" , df_O3_2023.describe(), "\n" , df_PM10_2023.describe(), "\n" , df_PM25_2023.describe(), "\n" , df_NO2_2024.describe(), "\n" , df_O3_2024.describe(), "\n" , df_PM10_2024.describe(), "\n" , df_PM25_2024.describe())
print("missing values in each df:"+ "\n"+ "\n")
print(df_NO2_2023.isnull().sum(), "\n" , df_O3_2023.isnull().sum(), "\n" , df_PM10_2023.isnull().sum(), "\n" , df_PM25_2023.isnull().sum(), "\n" , df_NO2_2024.isnull().sum(), "\n" , df_O3_2024.isnull().sum(), "\n" , df_PM10_2024.isnull().sum(), "\n" , df_PM25_2024.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11951 entries, 0 to 11950
Data columns (total 27 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   Country                                                         11951 non-null  object 
 1   Air Quality Network                                             11951 non-null  object 
 2   Air Quality Network Name                                        11951 non-null  object 
 3   Air Quality Station EoI Code                                    11951 non-null  object 
 4   Air Quality Station Name                                        11951 non-null  object 
 5   Sampling Point Id                                               11951 non-null  object 
 6   Air Pollutant                                                   11951 non-null  object 
 7   Air Pollutant Description                        